In [ ]:
# Load the necessary packages
import pandas as pd
import plotly.express as px
# Read the data from url below which is csv file
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQskznkuYK6r1QJ6eb5Cr8OIra9VEnKXN33NL-bwAORj8qDDk8wJR1tfgfuHzYvlPgJbjx4kP4wAwua/pub?gid=0&single=true&output=csv"
df= pd.read_csv(url)
df.head()

In [ ]:
# Lets understand the data structure
print(df.info())
print(df.describe())

In [ ]:
px.area(df, x='Date', y='Close')
px.box(df, y='Close')

In [ ]:
### Boxcox transformation
from statsmodels.base.transform import BoxCox
bc= BoxCox()
df["Close"], lmbda =bc.transform_boxcox(df["Close"])
## Making data Prophet complaint
data= df[["Date", "Close"]]
data.columns=["ds", "y"]

In [ ]:
# Import Prophet
from fbprophet import Prophet
## Creating model parameters
model_param ={
    "daily_seasonality": False,
    "weekly_seasonality":False,
    "yearly_seasonality":True,
    "seasonality_mode": "multiplicative",
    "growth": "logistic"
}
model = Prophet(**model_param)
data['cap']= data["y"].max() + data["y"].std() * 0.05 
model.fit(data)
# Create future dataframe
future= model.make_future_dataframe(periods=365)
future['cap'] = data['cap'].max()
forecast= model.predict(future)

In [ ]:
model.plot_components(forecast);
model.plot(forecast);

In [ ]:
## Adding parameters and seasonality and events
model = Prophet(**model_param)
model= model.add_seasonality(name="monthly", period=30, fourier_order=10)
model= model.add_seasonality(name="quarterly", period=92.25, fourier_order=10)
model.add_country_holidays("US")
model.fit(data)
# Create future dataframe
future= model.make_future_dataframe(periods=365)
future['cap'] = data['cap'].max()
forecast= model.predict(future)
model.plot_components(forecast);
model.plot(forecast);

In [ ]:
## Hyper parameter Tuning
import itertools
import numpy as np
from fbprophet.diagnostics import cross_validation, performance_metrics
param_grid={
    "daily_seasonality": [False],
    "weekly_seasonality":[False],
    "yearly_seasonality":[True],
    "growth": ["logistic"],
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5], # to give higher value to prior trend
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0] # to control the flexibility of seasonality components
}
# Generate all combination of parameters
all_params= [
            dict(zip(param_grid.keys(), v))
            for v in itertools.product(*param_grid.values())
]
print(all_params)

In [ ]:
# finding the best model

rmses= list ()
# go through each combinations
for params in all_params:
    m= Prophet(**params)
    
    m= m.add_seasonality(name= 'monthly', period=30, fourier_order=5)
    m= m.add_seasonality(name= "quarterly", period= 92.25, fourier_order= 10)
m.add_country_holidays(country_name="US")
    
    m.fit(data)
    
    df_cv= cross_validation(m, initial="730 days", period="365 days", horizon="365 days")
                            
    df_p= performance_metrics(df_cv, rolling_window=1)
                            
    rmses.append(df_p['rmse'].values[0])
                            
# find teh best parameters
best_params = all_params[np.argmin(rmses)]
                            
print("\n The best parameters are:", best_params)

In [ ]:
forecast["yhat"]=bc.untransform_boxcox(x=forecast["yhat"], lmbda=lmbda)
forecast["yhat_lower"]=bc.untransform_boxcox(x=forecast["yhat_lower"], lmbda=lmbda)
forecast["yhat_upper"]=bc.untransform_boxcox(x=forecast["yhat_upper"], lmbda=lmbda)
forecast.plot(x="ds", y=["yhat_lower", "yhat", "yhat_upper"])